In [1]:
import urllib
from sqlalchemy import create_engine

mssql_conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=RICHIE-IP330\MSSQLSERVER01;"
    "DATABASE=NEXT70;"
    "UID=Richard;"
    "PWD=richard123"
)

mssql_engine = create_engine(
    "mssql+pyodbc:///?odbc_connect=" + urllib.parse.quote_plus(mssql_conn_str),
    fast_executemany=True
)


In [2]:
import psycopg2

pg_conn = psycopg2.connect(
    host="13.229.51.249",
    database="TEST101",
    user="postgres",
    password="POSTGRES_PASSWORD_HERE",
    port=5432,
)
pg_conn.autocommit = False
pg_cur = pg_conn.cursor()


In [3]:
import pandas as pd

schema_df = pd.read_sql("""
SELECT
    TABLE_SCHEMA,
    TABLE_NAME,
    COLUMN_NAME,
    DATA_TYPE,
    CHARACTER_MAXIMUM_LENGTH
FROM INFORMATION_SCHEMA.COLUMNS
ORDER BY TABLE_NAME, ORDINAL_POSITION
""", mssql_engine)

In [4]:
type_map = {
    "int": "INTEGER",
    "bigint": "BIGINT",
    "smallint": "SMALLINT",
    "tinyint": "SMALLINT",
    "bit": "BOOLEAN",
    "decimal": "NUMERIC",
    "numeric": "NUMERIC",
    "float": "DOUBLE PRECISION",
    "real": "REAL",
    "money": "NUMERIC(19,4)",
    "datetime": "TIMESTAMP",
    "datetime2": "TIMESTAMP",
    "smalldatetime": "TIMESTAMP",
    "date": "DATE",
    "time": "TIME",
    "char": "CHAR",
    "nchar": "CHAR",
    "varchar": "VARCHAR",
    "nvarchar": "VARCHAR",
    "text": "TEXT",
    "ntext": "TEXT",
    "uniqueidentifier": "UUID"
}


In [5]:
tables_required = ["FMTRNVEW", "PSHP4VEW", "FMR01VEW"]

In [6]:
schema_df = schema_df[schema_df['TABLE_NAME'].isin(tables_required)]

In [7]:
schema_df.reset_index(inplace=True, drop=True)

In [8]:
schema_df

,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH
0,PMS,FMR01VEW,RESNUB,decimal,NaN
1,PMS,FMR01VEW,SRLNUB,decimal,NaN
2,PMS,FMR01VEW,SUBSRL,decimal,NaN
3,PMS,FMR01VEW,REFNUB,varchar,30.0
4,PMS,FMR01VEW,PRPCOD,varchar,3.0
...,...,...,...,...,...
180,PMS,PSHP4VEW,TOBQTY,decimal,NaN
181,PMS,PSHP4VEW,OTHCOV,decimal,NaN
182,PMS,PSHP4VEW,OTHAMT,decimal,NaN
183,PMS,PSHP4VEW,OTHQTY,decimal,NaN


In [9]:
for table in schema_df["TABLE_NAME"].unique():

    group = schema_df[schema_df["TABLE_NAME"] == table]

    cols = []

    for _, row in group.iterrows():
        pg_type = type_map.get(row.DATA_TYPE.lower(), "TEXT")
        col = f'"{row.COLUMN_NAME}" {pg_type}'
        cols.append(col)

    create_sql = f'''
    CREATE TABLE IF NOT EXISTS "{table}" (
        {", ".join(cols)}
    );
    '''

    pg_cur.execute(create_sql)

pg_conn.commit()


In [11]:
tables = schema_df['TABLE_NAME'].unique()

In [13]:
import io

tables = schema_df["TABLE_NAME"].unique()

for table in tables:

    query = f'SELECT * FROM PMS."{table}"'   # no schema

    print(f"Migrating {table}...")

    for chunk in pd.read_sql(query, mssql_engine, chunksize=100_000):

        buffer = io.StringIO()
        chunk.to_csv(buffer, index=False, header=False)
        buffer.seek(0)

        pg_cur.copy_expert(
            f'COPY "{table}" FROM STDIN WITH CSV',
            buffer
        )

        pg_conn.commit()


Migrating FMR01VEW...
Migrating FMTRNVEW...
Migrating PSHP4VEW...
